In [1]:
import sqlite3 
import yaml
import glob
import matplotlib.pyplot as plt
from plot_function import *
from sqlite_functions import *
import numpy as np
from pathlib import Path
from pybtex.database.input import bibtex

In [2]:
# Connect to database
# This will also create the database if not in the folder
connection = sqlite3.connect(Path('.') / 'litterature_data.db')
# Create a cursor
c = connection.cursor()

In [16]:
# Show existing tables
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())

[('papers',), ('samples',)]


# Create the tables

note that sql commands are usually capitalised

datatypes are : NULL INTEGER REAL TEXT BLOB

BLOB is everything else, for example images.

## Papers

In [4]:
#c.execute("DROP TABLE papers")
c.execute("""CREATE TABLE IF NOT EXISTS papers (
    paper_id text PRIMARY KEY,
    title text NOT NULL,
    authors text NOT NULL,
    year integer NOT NULL,
    journal text
    )""")
# Commit the changes in the database    
connection.commit()

## Samples 

In [15]:
# TODO : find if parsers exists for yml to sql
#c.execute("DROP TABLE samples")
# here the primary key will be an index with a int number incremented for each sample
c.execute("""CREATE TABLE IF NOT EXISTS samples (
    sample_id text PRIMARY KEY,
    type text,
    continuous_phase text,
    surfactant text,
    interface_type text, 
    solvent_viscosity integer,
    dispersed_phase text, 
    mean_radius float, 
    polydispersity float, 
    temperature float, 
    volume_fraction float,
    paper_id text, 
    FOREIGN KEY (paper_id) REFERENCES papers(paper_id)
    )""")
# Commit the changes in the database    
connection.commit()

In [14]:
list_samples = glob.glob('../raw_data/papers/**/samples_*.yml', recursive=True)
list_samples

['../raw_data/papers\\dekker2018scaling\\samples_dekker2018scaling.yml',
 '../raw_data/papers\\dinkgreve2015universal\\samples_dinkgreve2015universal.yml',
 '../raw_data/papers\\ghosh2019linear\\samples_ghosh2019linear.yml',
 '../raw_data/papers\\mason1996yielding\\samples_mason1996yielding.yml',
 '../raw_data/papers\\pamvouxoglou2021stress\\samples_pamvouxoglou2021stress.yml',
 '../raw_data/papers\\paredes2013rheology\\samples_paredes2013rheology.yml',
 '../raw_data/papers\\petekidis2004yielding\\samples_petekidis2004yielding.yml',
 '../raw_data/papers\\seth2011micromechanical\\samples_seth2011micromechanical.yml']

In [29]:
with open(list_samples[0]) as f:
        # use safe_load instead of load
        samples = list(yaml.load_all(f, Loader=yaml.FullLoader))

{'constant_parameters': {'type': 'emulsion',
  'continuous_phase': 'water',
  'surfactant': 'SDS',
  'solvent_viscosity': 0.00089,
  'dispersed_phase': 'castor oil',
  'mean_radius': 3.4e-06,
  'polydispersity': 20,
  'temperature': 25,
  'in_paper': 'dekker2018scaling',
  'in_figure': 'dekker2018scaling_fig2a'}}

In [31]:
paper = samples[0]['constant_parameters']['in_paper']

## Experiments

In [56]:
# TODO : use Marco's code

## Experiments type

# Populate tables 

## Populate papers from bib file

- open the bibtex file papers.bib
- list all existing entries
- if one of them is not in the papers table already, adds it.

In [5]:
# open the bibtex file
bibdata = bibtex.Parser().parse_file(Path('..') / 'raw_data' / 'papers.bib')

In [6]:
# get the papers ids that are already in the papers table
paper_ids = query_column('papers', 'paper_id')

In [7]:
list_paper_to_database = []
# loop through the references
for bib_id in bibdata.entries:
    b = bibdata.entries[bib_id].fields
    if bib_id not in paper_ids:
        try:
            title = b["title"]
            journal = b["journal"]
            year = b["year"]
            #deal with multiple authors
            authors = []
            for author in bibdata.entries[bib_id].persons["author"]:
                authors += [' '.join(author.first_names + author.last_names)]
            authors = ', '.join(authors)
            list_paper_to_database += [(bib_id, title, authors, year, journal)]
        except KeyError:
            continue

In [8]:
# Put that list into the database
c.executemany("INSERT INTO papers VALUES (?,?,?,?,?)", (list_paper_to_database))
# Then it needs to be commited to the database
connection.commit()